In [6]:
import logging
import cimpy
from pathlib import Path

# Configure logging (optional)
logging.basicConfig(filename='importCIGREMV.log', level=logging.INFO, filemode='w')

# Get absolute path to your XML file
xml_file_path = Path("./LVN/20250313T1045Z_RT_Own_TP_000.xml").resolve()

# Put it in a list because cim_import expects an iterable of file paths
xml_files = [str(xml_file_path)]

# Import the file using cim_import
import_result = cimpy.cim_import(xml_files, "cgmes_v2_4_15")

# For demonstration, print objects of certain CIM classes
results_to_show = ["ACLineSegment", "PowerTransformer", "EnergyConsumer"]
for key, value in import_result['topology'].items():
    if value.__class__.__name__ in results_to_show:
        print(value)

In [7]:
import logging
import cimpy
from pathlib import Path

# 1) Configure logging (optional)
logging.basicConfig(filename='importCIGREMV.log', level=logging.INFO, filemode='w')

# 2) Provide the path to your file
xml_file_path = Path("./LVN/20250313T1045Z_RT_Own_TP_000.xml").resolve()

# 3) cimpy needs a list of file paths
xml_files = [str(xml_file_path)]

# 4) Import your CGMES data
#    The string "cgmes_v2_4_15" tells cimpy which CIM profile definitions to use.
import_result = cimpy.cim_import(xml_files, "cgmes_v2_4_15")

# 5) Print the objects that match these CIM classes
#    You can add or remove other classes as needed.
classes_to_show = ["TopologicalNode", "Terminal"]
topology_dict = import_result.get('topology', {})  # 'topology' typically holds the topology profile objects

for rdf_id, obj in topology_dict.items():
    if obj.__class__.__name__ in classes_to_show:
        print(f"{rdf_id}: {obj}")

_0100b0e5-3cad-59ca-8ab9-babb2d1c6b83: class=TopologicalNode
DiagramObjects='list'
description=''
energyIdentCodeEic=''
mRID='_0100b0e5-3cad-59ca-8ab9-babb2d1c6b83'
name='SA'
shortName=''
AngleRefTopologicalIsland=None
BaseVoltage=None
ConnectivityNodeContainer=None
ConnectivityNodes='list'
ReportingGroup=None
SvInjection=None
SvVoltage=None
Terminal='list'
TopologicalIsland=None
boundaryPoint=False
fromEndIsoCode=''
fromEndName=''
fromEndNameTso=''
toEndIsoCode=''
toEndName=''
toEndNameTso=''

_01dfcc08-06f2-5852-8996-84ddb73d28fa: class=TopologicalNode
DiagramObjects='list'
description=''
energyIdentCodeEic=''
mRID='_01dfcc08-06f2-5852-8996-84ddb73d28fa'
name='SA'
shortName=''
AngleRefTopologicalIsland=None
BaseVoltage=None
ConnectivityNodeContainer=None
ConnectivityNodes='list'
ReportingGroup=None
SvInjection=None
SvVoltage=None
Terminal='list'
TopologicalIsland=None
boundaryPoint=False
fromEndIsoCode=''
fromEndName=''
fromEndNameTso=''
toEndIsoCode=''
toEndName=''
toEndNameTso=''



In [10]:
len(topology_dict)

845

In [8]:
for rdf_id, obj in topology_dict.items():
    if isinstance(obj, cimpy.classes_cim16.TopologicalNode):
        print(f"{rdf_id}, name: {obj.name}, baseVoltage: {obj.BaseVoltage}")

AttributeError: module 'cimpy' has no attribute 'classes_cim16'

In [12]:
import cimpy
import logging
from pathlib import Path
from collections import defaultdict

logging.basicConfig(level=logging.INFO)

# 1) Paths to your multiple CGMES files (EQ, TP, SSH, SV).
xml_files = [
    "./LEW/20250313T1045Z_RT_Own_EQ_000.xml",
    "./LEW/20250313T1045Z_RT_Own_TP_000.xml",
    "./LEW/20250313T1045Z_RT_Own_SSH_000.xml",
    "./LEW/20250313T1045Z_RT_Own_SV_000.xml"
]
xml_files = [str(Path(f).resolve()) for f in xml_files]

# 2) Import all profiles together so cimpy can build a complete model
import_result = cimpy.cim_import(xml_files, "cgmes_v2_4_15")

# 3) Typically, 'topology' profile data contains TopologicalIsland and TopologicalNode objects.
topology_dict = import_result.get("topology", {})

# 4) Identify all TopologicalIsland objects
#    We'll collect them in a dict keyed by RDF ID -> object
islands = {}
for rdf_id, obj in topology_dict.items():
    if obj.__class__.__name__ == "TopologicalIsland":
        islands[rdf_id] = obj

print(f"Number of topological islands (grids): {len(islands)}")

# 5) Count TopologicalNodes by the island to which they belong
#    Each TopologicalNode has an attribute: TopologicalNode.TopologicalIsland
#    If that attribute references one of our islands, we can group accordingly.
island_node_map = defaultdict(int)  # island_rdf_id -> number_of_nodes

for rdf_id, obj in topology_dict.items():
    if obj.__class__.__name__ == "TopologicalNode":
        # Check if this node references a TopologicalIsland
        island_ref = getattr(obj, "TopologicalIsland", None)  # sometimes called .TopologicalIsland or .island
        if island_ref:
            island_node_map[island_ref.rdfID] += 1
        else:
            # Some nodes might not be linked to an island if the file is partial or incomplete
            island_node_map["(No Island Reference)"] += 1

# 6) Print the node counts per island
for island_rdf_id, node_count in island_node_map.items():
    print(f"Island: {island_rdf_id}, TopologicalNodes: {node_count}")


Number of topological islands (grids): 1


AttributeError: 'TopologicalIsland' object has no attribute 'rdfID'

In [13]:
import logging
import cimpy
from pathlib import Path

# Optional: configure logging
logging.basicConfig(filename='importCIGREMV.log', level=logging.INFO, filemode='w')

# Paths to all relevant CGMES profile files
# (Equipment, Topology, SSH, SV, etc. if you have them)
xml_files = [
    "./LEW/20250313T1045Z_RT_Own_EQ_000.xml",
    "./LEW/20250313T1045Z_RT_Own_TP_000.xml",
    "./LEW/20250313T1045Z_RT_Own_SSH_000.xml",
    "./LEW/20250313T1045Z_RT_Own_SV_000.xml"
]
xml_files = [str(Path(f).resolve()) for f in xml_files]

# Import all files with cimpy
import_result = cimpy.cim_import(xml_files, "cgmes_v2_4_15")

# Retrieve objects from the 'topology' profile (which typically contains TopologicalNodes)
topology_dict = import_result.get('topology', {})

# Count all objects whose class name is "TopologicalNode"
total_nodes = sum(1 for obj in topology_dict.values() if obj.__class__.__name__ == "TopologicalNode")

print(f"Total number of TopologicalNodes: {total_nodes}")

Total number of TopologicalNodes: 845
